<a href="https://colab.research.google.com/github/AmiriChaima/Argument-detection/blob/main/Premise_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q transformers
! pip install -q datasets
! pip install -q scikit-learn
! pip install -q nltk
! pip install -q imbalanced-learn
! pip install -q optuna
! pip install -q transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.5 MB/s eta 0:00:00


In [2]:
import transformers
print(transformers.__version__)

import datasets
print(datasets.__version__)

import sklearn
print(sklearn.__version__)

import nltk
print(nltk.__version__)

import imblearn
print(imblearn.__version__)

4.33.3
2.14.5
1.2.2
3.8.1
0.10.1


In [3]:

# set model as required (a whole list of possible pre-trained Transformer models here: https://huggingface.co/transformers/pretrained_models.html)
model_checkpoint = "bert-base-uncased"
batch_size = 4

# required to use the inbuilt Transformers Trainer class
from datasets import load_dataset, load_metric

# set metric as required. Available inbuilt metrics (extend the datasets Metric class) are here: https://github.com/huggingface/datasets/tree/master/metrics. Can write your own custom metric too.
metric = load_metric('matthews_correlation')

# uncomment the below to print selected metric details
# print(metric)

<ipython-input-3-a6493723766e>:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('matthews_correlation')


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# read CSVs
url = 'https://raw.githubusercontent.com/AmiriChaima/MyData/main/data/Premise_Data.csv'


# read as pandas DataFrame (to be converted to dataset later)
df= pd.read_csv(url, skip_blank_lines=True, encoding='ISO-8859-1', delimiter=";")
df

,ID,Num_Doc,Statement,Type
0,305:11,Atom_902,the bottom left corner of the tabs are missing...,Premise: Specific usage experience
1,305:15,Atom_902,"Tabs were redesigned and shipped,",Premise: Evidence that the issue is or can be ...
2,306:8,Atom_995,Maybe dupes on labels is better with a `consol...,Premise: Evidence that the issue is or can be ...
3,307:14,Atom_1159,whether it's an inability to manage panes when...,Premise: Specific usage experience
4,308:7,Atom_1708,![screen shot 2014-03-05 at 10 11 02 pm](https...,Premise: Visual content or supporting file
...,...,...,...,...
793,607:5,VsCode_117499,https://github.com/microsoft/vscode/blob/7222d...,Premise: Visual content or supporting file
794,607:9,VsCode_117499,![TreeviewGlyphs](https://user-images.githubus...,Premise: Visual content or supporting file
795,607:10,VsCode_117499,"For example, the first entry in this treeview ...",Premise: Specific usage experience
796,608:7,VsCode_122839,"""Search include pattern"" and ""Search exclude p...",Premise: Clarification/ providing additional i...


In [5]:
possible_labels = df.Type.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict
df['Type'] = df.Type.replace(label_dict)



In [6]:
print("\n\nOriginal dataframe\n")
df=df.drop(columns=['ID','Num_Doc'],axis=1)
df = df.rename(columns={'Type': 'label','Statement':'text'})





Original dataframe



In [7]:
## 5-fold cross cv

from sklearn.model_selection import KFold, StratifiedKFold

n = 5
skf = StratifiedKFold(n_splits=n)
x = df["text"].to_numpy()
y = df["label"].to_numpy()

folds = []

for train_index, test_index in skf.split(x,y):
  x_train, y_train = x[train_index], y[train_index]
  x_test, y_test = x[test_index], y[test_index]

  train_d = pd.DataFrame({"text":x_train, "label":y_train})
  train_df, val_df = train_test_split(train_d, test_size=0.2, stratify=train_d["label"])
  test_df = pd.DataFrame({"text":x_test, "label":y_test})

  train = train_df.reset_index(drop=True)
  eval = val_df.reset_index(drop=True)
  test = test_df.reset_index(drop=True)

  folds.append([train, eval, test])


In [8]:

"""### EDA implementation"""

# imports for EDA
import random
from random import shuffle
random.seed(1)

import re

import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet

#stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our',
			'ours', 'ourselves', 'you', 'your', 'yours',
			'yourself', 'yourselves', 'he', 'him', 'his',
			'himself', 'she', 'her', 'hers', 'herself',
			'it', 'its', 'itself', 'they', 'them', 'their',
			'theirs', 'themselves', 'what', 'which', 'who',
			'whom', 'this', 'that', 'these', 'those', 'am',
			'is', 'are', 'was', 'were', 'be', 'been', 'being',
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at',
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after',
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again',
			'further', 'then', 'once', 'here', 'there', 'when',
			'where', 'why', 'how', 'all', 'any', 'both', 'each',
			'few', 'more', 'most', 'other', 'some', 'such', 'no',
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
			'very', 's', 't', 'can', 'will', 'just', 'don',
			'should', 'now', '']

#cleaning up text
import re
import re

def get_only_chars(line):
    clean_line = ""

    # change this as required!
    # line = line.replace("’", "")
    # line = line.replace("'", "")
    line = line.replace("-", " ")  # replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")

    for char in line:
        # can set own range as required.
        if char in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.?!.;:\'\" ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +', ' ', clean_line)  # delete extra spaces
    if len(clean_line) > 0 and clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################


def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word):
		for l in syn.lemmas():
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'])
			synonyms.add(synonym)
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

# def random_deletion(words, p):

# 	#obviously, if there's only one word, don't delete it
# 	if len(words) == 1:
# 		return words

# 	#randomly delete words with probability p
# 	new_words = []
# 	for word in words:
# 		r = random.uniform(0, 1)
# 		if r > p:
# 			new_words.append(word)

# 	#if you end up deleting all words, just return a random word
# 	if len(new_words) == 0:
# 		rand_int = random.randint(0, len(words)-1)
# 		return [words[rand_int]]

# 	return new_words

def random_deletion(words, p):
    # Obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    # Randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    # If you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return new_words



########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

# def random_swap(words, n):
# 	new_words = words.copy()
# 	for _ in range(n):
# 		new_words = swap_word(new_words)
# 	return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        if not new_words or len(new_words) < 2:
            break  # Break the loop if new_words is empty or has less than 2 words
        new_words = swap_word(new_words)
    return new_words


def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

# def random_insertion(words, n):
# 	new_words = words.copy()
# 	for _ in range(n):
# 		add_word(new_words)
# 	return new_words

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        if not new_words:
            break  # Break the loop if new_words is empty
        add_word(new_words)
    return new_words

# def add_word(new_words):
# 	synonyms = []
# 	counter = 0
# 	while len(synonyms) < 1:
# 		random_word = new_words[random.randint(0, len(new_words)-1)]
# 		synonyms = get_synonyms(random_word)
# 		counter += 1
# 		if counter >= 10:
# 			return
# 	random_synonym = synonyms[0]
# 	random_idx = random.randint(0, len(new_words)-1)
# 	new_words.insert(random_idx, random_synonym)

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        if not new_words:
            return  # Return if new_words is empty
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)


########################################################################
# main data augmentation function
########################################################################

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word != '']
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug / 4) + 1

    # sr
    if (alpha_sr > 0):
        n_sr = max(1, int(alpha_sr * num_words))
        for _ in range(num_new_per_technique):
            a_words = synonym_replacement(words, n_sr)
            augmented_sentences.append(' '.join(a_words))

    # ri
    if (alpha_ri > 0):
        n_ri = max(1, int(alpha_ri * num_words))
        for _ in range(num_new_per_technique):
            a_words = random_insertion(words, n_ri)
            augmented_sentences.append(' '.join(a_words))

    # rs
    if (alpha_rs > 0):
        n_rs = max(1, int(alpha_rs * num_words))
        for _ in range(num_new_per_technique):
            a_words = random_swap(words, n_rs)
            augmented_sentences.append(' '.join(a_words))

    # rd
    if (p_rd > 0 and num_words > 0):  # Ensure that words is not empty
        for _ in range(num_new_per_technique):
            a_words = random_deletion(words, p_rd)
            augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    # trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    # append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences

"""EDA sentence generator wrapper"""

def gen_eda(orig_train_df, output_train_df, alpha_sr, alpha_ri, alpha_rs, alpha_rd, num_aug=9):

  """
  Parameters:
  orig_train_df: unaugmented train (pandas) DataFrame
  output_train_df: empty DataFrame
  alpha_sr = augmentation parameter for synonym replacement
  alpha_ri = augmentation parameter for random insertion
  alpha_rs = augmentation parameter for random swapping
  alpha_rd = augmentation parameter for random deletion
  num_aug = number of generated sentences per input sentence.
  (generates (int(num_aug/4) + 1) sentences per task, shuffles dataset and trims to num_aug generated per sentence.)

  Returns:
  output_train_df = pandas DataFrame containing sentences from the original DataFrame + newly generated sentences.
  Size of output_train_df = (size of orig_train_df)*(num_aug + 1)
  """
  for idx in orig_train_df.index:
    label = orig_train_df.loc[idx, 'label']
    sentence = orig_train_df.loc[idx, 'text']
    aug_sentences = eda(sentence, alpha_sr=alpha_sr, alpha_ri=alpha_ri, alpha_rs=alpha_rs, p_rd=alpha_rd, num_aug=num_aug)
    d = {}
    d["text"] = aug_sentences
    d["label"] = [label for _ in range(len(aug_sentences))]
    df = pd.DataFrame(d)
    output_train_df = pd.concat([output_train_df, df], ignore_index=True)

  print("Generated", num_aug, "sentences per input sentence with EDA")
  return output_train_df



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [9]:
from datasets import Dataset

print("\n\nMetric details")
print(metric)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# uncomment below to test
# tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

def tokenize_function(ds):
  return tokenizer(ds["text"], truncation=True)



Metric details
Metric(name: "matthews_correlation", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (list of int): Predicted labels, as returned by a model.
    references (list of int): Ground truth labels.
    sample_weight (list of int, float, or bool): Sample weights. Defaults to `None`.
Returns:
    matthews_correlation (dict containing float): Matthews correlation.
Examples:
    Example 1, a basic example with only predictions and references as inputs:
        >>> matthews_metric = datasets.load_metric("matthews_correlation")
        >>> results = matthews_metric.compute(references=[1, 3, 2, 0, 3, 2],
        ...                                     predictions=[1, 2, 2, 0, 3, 3])
        >>> print(round(results['matthews_correlation'], 2))
        0.54

    Example 2, the same example as above, but also including sample weights:
        >>> matthews_metric = datasets.load_metric("matthews_co

In [10]:
! pip install accelerate>=0.20.1


In [11]:
from imblearn.over_sampling import RandomOverSampler
# for random import undersampling, uncomment the below:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter



from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
metric_name = "matthews_correlation"

In [13]:
args = TrainingArguments(
    "Argumentation",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=7.571619168090581e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.025036151003312385,
    warmup_steps=52,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

# needed during the training/eval loop (Trainer class instance). If you pass in compute_metrics=None (the default), evaluation is done with prediction loss only (no metric optimization objective)
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)


for fold_i in range(len(folds)):
  print("\nFold", fold_i+1, "\n")

  train_df, eval_df, test_df = folds[fold_i][0], folds[fold_i][1], folds[fold_i][2]

  # ----------------- EDA ----------------------------
  # empty DataFrame
  output_train = pd.DataFrame(columns=["text", "label"])
  aug_train = gen_eda(train_df, output_train, 0.05, 0.05, 0.05, 0.05, 4)

  print("\n\n# of datapoints before augmentation:", len(train_df))
  print("# of datapoints after augmentation:", len(aug_train))


Fold 1 

Generated 4 sentences per input sentence with EDA


# of datapoints before augmentation: 510
# of datapoints after augmentation: 2550

Fold 2 

Generated 4 sentences per input sentence with EDA


# of datapoints before augmentation: 510
# of datapoints after augmentation: 2550

Fold 3 

Generated 4 sentences per input sentence with EDA


# of datapoints before augmentation: 510
# of datapoints after augmentation: 2550

Fold 4 

Generated 4 sentences per input sentence with EDA


# of datapoints before augmentation: 511
# of datapoints after augmentation: 2555

Fold 5 

Generated 4 sentences per input sentence with EDA


# of datapoints before augmentation: 511
# of datapoints after augmentation: 2555


In [14]:



  # aug_train is final ds

  ## NO BALANCING!!
  """### Class balancing: random oversampling"""


  train_text = aug_train["text"]
  train_labels = aug_train["label"]
  print("#\n\nLabel count before oversampling:", Counter(train_labels))

  # X_train_np, y_train_l = train_text.to_numpy(), train_labels.to_list()
  # X_train_np = X_train_np.reshape((-1, 1))

  # from sklearn.utils.multiclass import type_of_target - need this!
  # print(type_of_target(tr_list))

  # ros = RandomOverSampler(sampling_strategy="minority")
  # X_resampled, y_resampled = ros.fit_resample(X_train_np, y_train_l)

  # for random undersampling, uncomment the below:
  # rus = RandomUnderSampler(sampling_strategy="majority")
  # X_resampled, y_resampled = rus.fit_resample(X_train_np, y_train_np)

  # shape = (len(y_resampled),)
  # X_resampled = X_resampled.reshape(shape)

  # train_text = X_resampled
  # train_labels = y_resampled

  # print("Label count after oversampling:", Counter(train_labels))
  # print("Label count after undersampling:", Counter(train_labels))

  # train = pd.DataFrame({'text': train_text, 'label': train_labels})

  # print("\n\nTrain dataset after augmentation and oversampling\n")
  # print(train.head())

  # convert to dataset for easier Transformer integration

  # aug_train!
  train_ds = Dataset.from_pandas(aug_train)
  eval_ds = Dataset.from_pandas(eval_df)
  test_ds = Dataset.from_pandas(test_df)

  # train_ds[0]



#

Label count before oversampling: Counter({2: 960, 0: 570, 1: 410, 3: 260, 5: 220, 4: 135})


In [15]:
# ------------- TOKENIZATION ----------

  # tokenize datasets (preprocessing)
encoded_train_ds = train_ds.map(tokenize_function, batched=True)
encoded_eval_ds = eval_ds.map(tokenize_function, batched=True)
encoded_test_ds = test_ds.map(tokenize_function, batched=True)

print("\n\nEncoded train dataset sample\n")
print(encoded_train_ds[0])
print("\n\n")

trainer = Trainer(
    model,
    args=args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
  )


trainer.train()
trainer.evaluate()



Map:   0%|          | 0/2555 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/159 [00:00<?, ? examples/s]



Encoded train dataset sample

{'text': 'This is because the width of the tab tabs held constant until the mouse leaves the tab area so if I close a tab the next tab moves to occupy the exact same space that the previous tab did with the "x" in the same allowing position me to quickly close a large number of is by simply clicking repeatedly in the same spot.', 'label': 0, 'input_ids': [101, 2023, 2003, 2138, 1996, 9381, 1997, 1996, 21628, 21628, 2015, 2218, 5377, 2127, 1996, 8000, 3727, 1996, 21628, 2181, 2061, 2065, 1045, 2485, 1037, 21628, 1996, 2279, 21628, 5829, 2000, 11494, 1996, 6635, 2168, 2686, 2008, 1996, 3025, 21628, 2106, 2007, 1996, 1000, 1060, 1000, 1999, 1996, 2168, 4352, 2597, 2033, 2000, 2855, 2485, 1037, 2312, 2193, 1997, 2003, 2011, 3432, 22042, 8385, 1999, 1996, 2168, 3962, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,1.029196,0.466896
2,No log,1.533897,0.505017
3,No log,1.703811,0.512353


{'eval_loss': 1.7038109302520752,
 'eval_matthews_correlation': 0.5123530263515108,
 'eval_runtime': 0.5664,
 'eval_samples_per_second': 226.002,
 'eval_steps_per_second': 56.5,
 'epoch': 3.0}

In [16]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import numpy as np
# Make predictions using the model
predictions, labels, metrics = trainer.predict(encoded_test_ds)

# Calculate predicted labels based on prediction probabilities
predicted_labels = []
for logits in predictions:
    predicted_label = np.argmax(logits)
    predicted_labels.append(predicted_label)

# Print the predicted labels and metrics
print("\nPREDICTED LABELS:\n", predicted_labels)
print("\nMetrics:\n", metrics)

# Print the original test dataset and expected labels
print("\n\n-----\nOriginal test dataset:")
print("\nTEXT")
count = 1
for i in test_df["text"]:
    print(count, ": ", i, "\n")
    count += 1
truth_labels = [_ for _ in test_df["label"]]
print("\nEXPECTED LABEL")
print(truth_labels)

# Print class labels distribution
print("\nClass labels distribution:", test_df['label'].value_counts(normalize=False))

# Calculate and print F1-score, precision, and recall
f1 = f1_score(truth_labels, predicted_labels, average='weighted')
precision = precision_score(truth_labels, predicted_labels, average='weighted')
recall = recall_score(truth_labels, predicted_labels, average='weighted')

print("\nWeighted F1 Score:", f1)
print("Weighted Precision:", precision)
print("Weighted Recall:", recall)

# Generate a classification report
print("###################################\nALL SCORES\n")
print("\nclassification report\n")
print(classification_report(truth_labels, predicted_labels))

# predictions, labels, metrics = trainer.predict(encoded_test_ds)
# # print("\nPredictions logits:\n", predictions)
# list_predictions_logits = [_ for _ in predictions]
# predicted_labels = [_ for _ in range(len(list_predictions_logits))]
# for i in range(len(list_predictions_logits)):
#     if list_predictions_logits[i][0]>list_predictions_logits[i][1]:
#         predicted_labels[i] = 0
#     else:
#         predicted_labels[i] = 1

# print("\nPREDICTED LABELS:\n", predicted_labels)
# print("\nMetrics:\n", metrics)

# print("\n\n-----\nOriginal test dataset:")
# print("\nTEXT")
# count = 1
# for i in test_df["text"]:
#   print(count, ": ", i, "\n")
#   count += 1
#   truth_labels = [_ for _ in test_df["label"]]
# print("\nEXPECTED LABEL")
# print(truth_labels)

# print("\nClass labels distribution:", test_df['label'].value_counts(normalize=False))


# print("###################################\nALL SCORES\n")

# print("\nclassification report\n")
# print(classification_report(truth_labels, predicted_labels))


PREDICTED LABELS:
 [1, 0, 5, 0, 1, 3, 1, 3, 1, 4, 3, 2, 1, 2, 2, 0, 1, 5, 2, 2, 5, 0, 2, 1, 2, 2, 5, 1, 2, 1, 5, 2, 5, 2, 2, 0, 0, 0, 2, 1, 0, 5, 2, 2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 3, 5, 0, 5, 2, 5, 1, 5, 2, 5, 2, 2, 2, 3, 2, 0, 0, 3, 1, 3, 0, 2, 2, 0, 0, 0, 2, 0, 5, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 0, 0, 5, 3, 2, 1, 2, 1, 0, 0, 3, 2, 2, 3, 0, 1, 1, 2, 2, 1, 2, 2, 2, 5, 2, 1, 1, 2, 1, 2, 0, 1, 3, 0, 0, 2, 5, 2, 0, 0, 2, 1, 2, 0, 2, 5, 4, 0, 2, 2, 2, 0, 0, 5, 1, 0, 0, 1, 1, 0, 2, 2, 0, 5, 3]

Metrics:
 {'test_loss': 1.7198574542999268, 'test_matthews_correlation': 0.523342515873108, 'test_runtime': 0.7334, 'test_samples_per_second': 216.786, 'test_steps_per_second': 54.537}


-----
Original test dataset:

TEXT
1 :   This would benefit both web-embedded Monaco and in VS Code visually and accessibility-wise. 

2 :  Like the title says, when nothing is selected, copying shouldn't copy a blank character. 

3 :  because there is no recyclebin for a USB Disk. 

4 :  so that we can tell the 